In [ ]:
print("Hello world")

In [ ]:
import os
import tarfile
from six.moves import urllib

In [ ]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = "datasets/housing"
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path=os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
housing= load_housing_data()

In [ ]:
housing.head()

In [ ]:
housing.info()

In [ ]:
housing.describe()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
print(len(train_set), len(test_set))

In [ ]:
import numpy as np

In [ ]:
housing['income_cat']=np.ceil(housing['median_income']/1.5)

In [ ]:
housing['income_cat'].where(housing['income_cat']<5,5.0,inplace=True)

In [ ]:
housing['income_cat'].value_counts()

In [ ]:
housing.drop(['income_cat'])

In [ ]:
housing.head()

In [ ]:
for set in (train_set,test_set):
    set.drop(["income_cat"], axis=1, inplace=True)

In [ ]:
housing.head()

In [ ]:
housing.drop(['income_cat'],axis=1)

In [ ]:
housing.head()

In [ ]:
housing=housing.drop(['income_cat'],axis=1)

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

In [ ]:
housing.plot(kind='scatter',x='longitude',y='latitude',alpha=0.1)

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
corr_matrix=housing.corr()

In [ ]:
corr_matrix.sort_values(ascending=False)

In [ ]:
corr_matrix.sort_values(by,ascending=False)

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
corr_matrix.value_counts()

In [ ]:
housing=train_set.drop(['median_house_value'],axis=1)
housing_labels=train_set['median_house_value'].copy()

In [ ]:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median) 

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
housing_cat=housing['ocean_proximity']
housing_cat_encoded=encoder.fit_transform(housing_cat)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()
housing_cat_1hot=encoder.fit_transform(housing_cat_encoded.reshape(-1,1))

In [ ]:
print(housing_cat_1hot.toarray())

In [72]:
housing['ocean_proximity1hot']=housing_cat_1hot.toarray()

In [78]:
housing=housing.join(pd.DataFrame(housing_cat_1hot.toarray()))

In [80]:
housing.fillna(-99999,inplace=True)

In [82]:
housing=housing.drop(['ocean_proximity'],axis=1)

In [89]:
housing.columns

Index([         'longitude',           'latitude', 'housing_median_age',
              'total_rooms',     'total_bedrooms',         'population',
               'households',      'median_income',                    0,
                          1,                    2,                    3,
                          4, 'median_house_value'],
      dtype='object')

In [90]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
X=housing.drop(['median_house_value'],axis=1)
y=housing['median_house_value']

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2)

In [92]:
model.fit(X_train, y_train)

/home/meetr/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


LinearRegression()

In [93]:
confidence=model.score(X_test, y_test)
print(confidence)

0.6243361377022907


/home/meetr/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [94]:
import pickle

In [96]:
with open('california_housing_linearRG.pickle','wb')as files:
    pickle.dump(model,files)

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
num_pipeline = Pipeline([
('selector', DataFrameSelector(num_attribs)),
('imputer', Imputer(strategy="median")),
('attribs_adder', CombinedAttributesAdder()),
('std_scaler', StandardScaler()),
])
cat_pipeline = Pipeline([
('selector', DataFrameSelector(cat_attribs)),
('label_binarizer', LabelBinarizer()),
])
full_pipeline = FeatureUnion(transformer_list=[
("num_pipeline", num_pipeline),
("cat_pipeline", cat_pipeline),
])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
num_pipeline = Pipeline([
('imputer', imputer(strategy="median")),
('attribs_adder', CombinedAttributesAdder()),
('std_scaler', StandardScaler()),
])
housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)